In [ ]:
!pip install tensorflow-gpu==1.15.4

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
print(tf. __version__) 
embed = hub.Module("https://tfhub.dev/google/elmo/2")
print(len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import os

In [ ]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/Colab Datasets/super_combine.csv", encoding='latin-1')

# data = pd.read_csv("super_combine.csv", encoding='latin-1')
# data = pd.read_excel('punny2.xlsx')
data.columns =["text", "category"]
print(data.head(5))
# print(data.describe())

In [ ]:
from sklearn.preprocessing import LabelEncoder
df3 = pd.DataFrame()
df3["text"] = data["text"]
df3["label"] = LabelEncoder().fit_transform(data["category"])

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
df3['text'] = df3['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df3['text'].tail()

In [ ]:
!pip uninstall keras

In [ ]:
!pip install keras==2.3.0

In [ ]:
!pip uninstall python3

In [ ]:
from sklearn import preprocessing
import keras
y = list(df3['label'])
x = list(df3['text'])

en = preprocessing.LabelEncoder()
en.fit(y)

from keras.utils import np_utils
def encode(le_enc, labels):
    enc = le_enc.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(en, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return en.inverse_transform(dec)

x_enc = x
y_enc = encode(en, y)

x_train = x_enc[:52588]
y_train = y_enc[:52588]
print(len(x_train))
print(x_train[30459])

42627
Best luck.


Using TensorFlow backend.


In [ ]:
from sklearn.model_selection import train_test_split
x_train1, x_test, y_train1, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=1, random_state=88)

In [ ]:
pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(np.asarray(x_train), np.asarray(y_train), epochs=7, batch_size=16)
    model.save_weights('./elmo-model.h5')
    # data = pd.read_csv("/content/drive/My Drive/colab_ds/ds_super_fast.csv", encoding='latin-1')

In [ ]:
x_test = x_enc[52589:65736] #30460:30684 30685:30909 55911:56135 31810:32034 31135:31359 56361:56585 56586:56810
y_test = y_enc[52589:65736]
print(x_test)
# print(x_test[224])

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')
    predicts = model.predict(np.asarray(x_test), batch_size=16)

y_test = decode(en, y_test)
y_preds = decode(en, predicts)

In [ ]:
print(y_test)
print (y_preds)
# Calculate accuracy from the above two values
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, y_preds))

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')
    pred_lbl = (model.predict(np.asarray(x_test)) > 0.5).astype("int32") #decode(en, x_test)
    pd.DataFrame(pred_lbl).to_csv('output2.csv', index=False)

In [ ]:
import pandas as pd
messages = pd.read_csv("output2b.csv", encoding='latin-1')
messages.columns = ["labels"]
print (messages.tail(3))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, messages["labels"]))
print(metrics.confusion_matrix(y_test, messages["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(y_test, messages["labels"])))